In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import math
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode            # What you'd normally run
#from learntools.geospatial.tools import geocode # Just for this exercise
import requests
import urllib.parse

import folium 
from folium import Marker
from folium.plugins import HeatMap, MarkerCluster

from learntools.core import binder
binder.bind(globals())

import numpy as np

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
np.random.seed(0) 
import geopy
from geopy.geocoders import Nominatim

nom=Nominatim(user_agent="wineTest")
geopy.geocoders.options.default_user_agent = "wineTest"
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
wineFP="../input/wine-reviews/winemag-data-130k-v2.csv"
wineData=pd.read_csv(wineFP)
wineData.head()
#removing redundant/lost information
wineData = wineData.loc[~np.isnan(wineData["price"])]
wineData = wineData[pd.notnull(wineData['country'])]
wineData = wineData[pd.notnull(wineData['province'])]
#limits the use of US states as output
wineData = wineData.loc[~(wineData.country.isin(['US'])&(wineData.region_1.isnull()|(wineData.region_1==wineData.province)))]
#removes all counties that were "other" to prevent geolocating to austria Idk
wineData = wineData.loc[~(wineData.country=='Other')]

#Drops 25% of data that lack critical values

In [ ]:
#test
nom.geocode({'country': 'DE', 'city': 'Erlangen', 'postalcode': '91052',
                                'street': 'Nürnberger Straße 6'})

In [ ]:
#reassignment
bruh=wineData.iloc[65000:69999]
len(bruh)

In [ ]:
#location spe
bruh['province'] = bruh['province'].fillna(bruh['country'])
bruh['region_1'] = bruh['region_1'].fillna(bruh['province'])
bruh['region_2'] = bruh['region_2'].fillna(bruh['region_1'])

In [ ]:
def my_geocoder(row):
    try:
        point = geocode(row, provider='nominatim').geometry.iloc[0]
        return pd.Series({'Latitude': point.y, 'Longitude': point.x, 'geometry': point})
    except:
        #return None
        return pd.Series({'Longitude': 0})
wineData.head()
#wineData[['Latitude', 'Longitude', 'geometry']] = (wineData.head(200)).apply(lambda x: my_geocoder(x['designation']), axis=1)
bruh[['Latitude', 'Longitude', 'geometry']]= (wineData.iloc[65000:69999]).apply(lambda x: my_geocoder(x['region_2']), axis=1)



In [ ]:
bruh.head()

In [ ]:
print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(bruh["Latitude"])) / len(bruh)) * 100))

# Drop  that were not successfully geocoded
bruh = bruh.loc[~np.isnan(bruh["Latitude"])]
bruh = gpd.GeoDataFrame(bruh, geometry=bruh.geometry)
bruh.crs = {'init': 'epsg:4326'}
bruh.head()

#removes on average 8% of data

In [ ]:
# Create a map
m = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)

# Add points to the map
#for idx, row in bruh.iterrows():
#    Marker([row['Latitude'], row['Longitude']], popup=row['designation']).add_to(m)

HeatMap(data=sum1[['Latitude', 'Longitude']], radius=10).add_to(m)

# Display the map
m

In [ ]:
bruh.to_csv('70000.csv',index=False)

Data Muting

In [ ]:
sum1=pd.read_csv("./65k.csv")
b=pd.read_csv("./70000.csv")

sum2=pd.concat([sum1,b])
sum2.info()
sum2.to_csv('70k.csv',index=False)

In [ ]:
sum1=pd.read_csv("../input/70kresults/70k.csv")